In [1]:
import pandas as pd
import urllib.request, json
import logging

In [3]:
logging.basicConfig(filename='../data/log/oac_geo.log', filemode='w', level=logging.DEBUG, format='%(asctime)s %(message)s')

In [3]:
df = pd.read_excel('../data/input/FY24 Older Adult Centers (11-20-23).xlsx', sheet_name='SC List by CMs 8a')

In [14]:
df['geoclient'] = (df['Program Address'] + ", " + df['Borough'] + ", " + df['Zip Code'].astype(str))

In [16]:
def json_get(url, row):
    try: 
        hdr ={
        # Request headers
        'Cache-Control': 'no-cache',
        'Ocp-Apim-Subscription-Key': '2fc22446e7af4d4eaa50d383ad846872',
        }

        req = urllib.request.Request(url, headers=hdr)

        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)
        logging.info(f'{row.Index}: {response.getcode()}')
        json_data = json.loads(response.read())
    except Exception as e:
        logging.info(e)
        json_data = {}
    return json_data

In [23]:
def get_lat_lon(df):
    lat = []
    lon = []
    for row in df.itertuples():
        ### figure out which request type and url to use
        # if cross_street_name exists, use that
        type = "search"
        url = f'https://api.nyc.gov/geo/geoclient/v1/search.json?input={row.geoclient}'
        json_data = json_get(url, row)

        ### depending on the request type, json will have different fields
        try:
            lat.append(json_data['results'][0]['response']['latitude'])
            lon.append(json_data['results'][0]['response']['longitude'])
        except Exception as e:
            #logging.info(e.args)
            try:
                message = json_data['status']
                logging.info(f'{row.Index}: {message}')
            except Exception as e:
                logging.info(f'{row.Index}: {e}')
            lat.append('NA')
            lon.append('NA')
    return lat, lon

In [36]:
df['geoclient'] = df['geoclient'].replace(' ', '%20', regex=True)
df['geoclient'] = df['geoclient'].replace('\n', '%20', regex=True)

In [37]:
df['lat'], df['lon'] = get_lat_lon(df)

In [38]:
df.to_csv('../data/output/df_geo.csv', index=False)